# DnCNN for Image Denoising


Beyond a Gaussian Denoiser: Residual Learning of Deep CNN for Image Denoising
https://arxiv.org/pdf/1608.03981.pdf

Download and unzip dataset

In [2]:
from pathlib import Path

if not Path("./small_fastmri_pd_3t").is_dir():
    !gdown --id "1y78Ad6WwQpMGtxfEZlp97A0iV98kAiJN"
    !unzip -q small_fastmri_pd_3t.zip && rm small_fastmri_pd_3t.zip
    
if not Path("./dncnn-noiseless.pth").is_file():
    !gdown --id "1azlqmuIkdhcsMQJL_YObF4sEe83D8J8N" 
# Noiseless model weights: https://drive.google.com/file/d/1azlqmuIkdhcsMQJL_YObF4sEe83D8J8N/view?usp=sharing
# Gaussian model weights: 
# Salt&Pepper model weights: 
# Gaussian + Salt&Pepper model weights: 

# TO DO
# 1. Load noiseless and train 10 more epoches with noises, 3 types


In [3]:
!nvidia-smi

Mon May 17 13:19:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  On   | 00000000:17:00.0 Off |                  N/A |
| 90%   84C    P2   239W / 280W |   9230MiB / 11178MiB |     96%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  On   | 00000000:65:00.0 Off |                  N/A |
|  0%   45C    P8    12W / 280W |     12MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [4]:
import os
import sys
import numpy as np
import h5py
import pylab as plt
import torch

import pytorch_lightning as pl
from pytorch_lightning.callbacks import Callback, ModelCheckpoint
from k_space_reconstruction.nets.dncnn import DnCNNModule
from k_space_reconstruction.datasets.fastmri import FastMRITransform, FastMRIh5Dataset, RandomMaskFunc
from k_space_reconstruction.utils.metrics import pt_msssim, pt_ssim, ssim, nmse, psnr
from k_space_reconstruction.utils.loss import l1_loss, compund_mssim_l1_loss
from k_space_reconstruction.utils.kspace import spatial2kspace, kspace2spatial

print('Available GPUs: ', torch.cuda.device_count())

Available GPUs:  3


# Dataset initialization

In [5]:
transform = FastMRITransform(
    RandomMaskFunc([0.08], [4]),
    noise_level=1000,
    noise_type='none'
)

train_dataset = FastMRIh5Dataset('small_fastmri_pd_3t/train.h5', transform)
val_dataset = FastMRIh5Dataset('small_fastmri_pd_3t/val.h5', transform)
train_generator = torch.utils.data.DataLoader(train_dataset, batch_size=1, num_workers=12)
val_generator = torch.utils.data.DataLoader(val_dataset, batch_size=1, num_workers=12)

/home/ekuzmina/dl-bia-project/k_space_reconstruction/datasets/fastmri.py:186: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.hf = h5py.File(hf_path)


# Model definition

In [6]:
net = DnCNNModule(
    dncnn_chans=64, 
    dncnn_depth=10, 
    criterion=compund_mssim_l1_loss, 
    verbose_batch=50, 
    optimizer='Adam',
    lr=1e-4,
    lr_step_size=5,
    lr_gamma=0.2,
    weight_decay=0.0
)

# Tensorboard logging

In [10]:
%load_ext tensorboard
%tensorboard --logdir logs/ --port 8123

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 8123 (pid 27897), started 1 day, 22:01:47 ago. (Use '!kill 27897' to kill it.)

# Init trainer

In [8]:
trainer = pl.Trainer(
    gpus=1, max_epochs=40,
    accumulate_grad_batches=32,
    terminate_on_nan=True, 
    default_root_dir='logs/DnCNN',
    callbacks=[
        pl.callbacks.ModelCheckpoint(
            save_last=True,
            save_top_k=4, 
            monitor='val_loss', 
            filename='{epoch}-{ssim:.4f}-{psnr:.4f}-{nmse:.5f}'
        ),
        pl.callbacks.LearningRateMonitor(logging_interval='epoch'),
        pl.callbacks.GPUStatsMonitor(temperature=True)
    ]
);

GPU available: True, used: True
TPU available: None, using: 0 TPU cores


# Train model

In [9]:
trainer.fit(net, train_dataloader=train_generator, val_dataloaders=val_generator)


  | Name             | Type                 | Params
----------------------------------------------------------
0 | net              | DnCNN                | 297 K 
1 | NMSE             | DistributedMetricSum | 0     
2 | SSIM             | DistributedMetricSum | 0     
3 | PSNR             | DistributedMetricSum | 0     
4 | ValLoss          | DistributedMetricSum | 0     
5 | TotExamples      | DistributedMetricSum | 0     
6 | TotSliceExamples | DistributedMetricSum | 0     
----------------------------------------------------------
297 K     Trainable params
0         Non-trainable params
297 K     Total params
1.189     Total estimated model params size (MB)


Epoch 0:  10%|▉         | 239/2437 [00:14<02:12, 16.54it/s, loss=0.227, v_num=8, val_loss=0.216]

Saving latest checkpoint...


MisconfigurationException: ModelCheckpoint(monitor='ssim') not found in the returned metrics: ['train_loss_step']. HINT: Did you call self.log('ssim', tensor) in the LightningModule?

# Test model
Load best checkpoint, inference on val dataset and save predictions to .h5 file in logs directory

In [8]:
!ls "logs/DnCNN/lightning_logs/version_2/checkpoints/"

'epoch=12-ssim=0.7514-psnr=27.9779-nmse=0.01845.ckpt'
'epoch=13-ssim=0.7516-psnr=27.9857-nmse=0.01842.ckpt'
'epoch=14-ssim=0.7517-psnr=27.9931-nmse=0.01838.ckpt'
'epoch=19-ssim=0.7509-psnr=27.9630-nmse=0.01854.ckpt'
 last.ckpt


In [7]:
net.net.load_state_dict(torch.load('last.ckpt'))
# net.eval()

<All keys matched successfully>

In [32]:
trainer.test(net, val_generator)

Testing: 100%|██████████| 395/395 [00:10<00:00, 37.02it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


[{}]

In [34]:
hf_pred = h5py.File('logs/DnCNN/2021-05-14 18:49:03.756824.h5')
hf_gt = h5py.File('small_fastmri_pd_3t/val.h5')

<ipython-input-34-40a733593629>:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  hf_pred = h5py.File('logs/DnCNN/2021-05-14 18:49:03.756824.h5')
<ipython-input-34-40a733593629>:2: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  hf_gt = h5py.File('small_fastmri_pd_3t/val.h5')


# Val metrics

In [35]:
ssim_vals = []
nmse_vals = []
psnr_vals = []
for k in hf_pred.keys():
    ks = hf_gt[k][:] * 1e6
    gt = np.stack([kspace2spatial(k) for k in ks])
    pred = hf_pred[k][:,0]
    ssim_vals.append(ssim(gt, pred))
    nmse_vals.append(nmse(gt, pred))
    psnr_vals.append(psnr(gt, pred))
ssim_vals = np.array(ssim_vals)
nmse_vals = np.array(nmse_vals)
psnr_vals = np.array(psnr_vals)

In [36]:
np.mean(ssim_vals), np.mean(nmse_vals), np.mean(psnr_vals)

(0.7450819779897695, 0.020334082331490425, 28.62007806339689)

## Saving Weights of the Model 

In [15]:
torch.save(net.net.state_dict(), 'dncnn-noiseless.pth')

# Continue Training with Noises

We take our **noiseless** pre-trained model weight and continue training with different types of noises.

- Gaussian noise, lvl: 400
- Salt&Pepper noise, lvl: 5e4
- Gaussian + Salt&Pepper noise, lvl: 400 + 5e4

### Gaussian Noise

In [7]:
net.net.load_state_dict(torch.load('dncnn-noiseless.pth'))
# net

<All keys matched successfully>

In [8]:
transform = FastMRITransform(
    RandomMaskFunc([0.08], [4]),
    noise_level=400,
    noise_type='normal'
)

train_dataset = FastMRIh5Dataset('small_fastmri_pd_3t/train.h5', transform)
val_dataset = FastMRIh5Dataset('small_fastmri_pd_3t/val.h5', transform)
train_generator = torch.utils.data.DataLoader(train_dataset, batch_size=1, num_workers=12)
val_generator = torch.utils.data.DataLoader(val_dataset, batch_size=1, num_workers=12)

In [ ]:
trainer = pl.Trainer(
    gpus=1, max_epochs=20,
    accumulate_grad_batches=32,
    terminate_on_nan=True, 
    default_root_dir='logs/DnCNN_gaussian',
    callbacks=[
        pl.callbacks.ModelCheckpoint(
            save_last=True,
            save_top_k=4, 
            monitor='val_loss', 
            filename='{epoch}-{ssim:.4f}-{psnr:.4f}-{nmse:.5f}'
        ),
        pl.callbacks.LearningRateMonitor(logging_interval='epoch'),
        pl.callbacks.GPUStatsMonitor(temperature=True)
    ]
);

trainer.fit(net, train_dataloader=train_generator, val_dataloaders=val_generator)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name             | Type                 | Params
----------------------------------------------------------
0 | net              | DnCNN                | 297 K 
1 | NMSE             | DistributedMetricSum | 0     
2 | SSIM             | DistributedMetricSum | 0     
3 | PSNR             | DistributedMetricSum | 0     
4 | ValLoss          | DistributedMetricSum | 0     
5 | TotExamples      | DistributedMetricSum | 0     
6 | TotSliceExamples | DistributedMetricSum | 0     
----------------------------------------------------------
297 K     Trainable params
0         Non-trainable params
297 K     Total params
1.189     Total estimated model params size (MB)


Epoch 0:   0%|          | 0/2437 [00:00<?, ?it/s]                     

/home/ekuzmina/anaconda3/envs/pytorch38/lib/python3.8/site-packages/pytorch_lightning/core/step_result.py:148: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  value = torch.tensor(value, device=device, dtype=torch.float)


Epoch 0:  84%|████████▍ | 2042/2437 [05:30<01:04,  6.17it/s, loss=0.0652, v_num=6, val_loss=0.126]
Validating: 0it [00:00, ?it/s]
Epoch 0:  96%|█████████▌| 2333/2437 [05:42<00:15,  6.81it/s, loss=0.0652, v_num=6, val_loss=0.126]


Epoch 1:  84%|████████▍ | 2042/2437 [04:37<00:53,  7.36it/s, loss=0.0467, v_num=6, val_loss=0.101]
Validating: 0it [00:00, ?it/s]
Epoch 1:  91%|█████████ | 2216/2437 [04:45<00:28,  7.77it/s, loss=0.0467, v_num=6, val_loss=0.101]


...

Epoch 4:  84%|████████▍ | 2042/2437 [04:01<00:46,  8.46it/s, loss=0.0408, v_num=6, val_loss=0.0825]
Validating: 0it [00:00, ?it/s]
Epoch 4:  90%|█████████ | 2194/2437 [04:06<00:27,  8.90it/s, loss=0.0408, v_num=6, val_loss=0.0825]


Epoch 5:  84%|████████▍ | 2042/2437 [04:33<00:52,  7.48it/s, loss=0.0404, v_num=6, val_loss=0.0814]
Validating: 0it [00:00, ?it/s]
Epoch 5:  88%|████████▊ | 2150/2437 [04:38<00:37,  7.73it/s, loss=0.0404, v_num=6, val_loss=0.0814]


Epoch 6:  84%|████████▍ | 2042/2437 [06:06<01:10,  5.56it/s, loss=0.0403, v_num=6, val_loss=0.0798]
Validating: 0it [00:00, ?it/s]
Validating:   0%|          | 0/395 [00:00<?, ?it/s]


Validating:  73%|███████▎  | 288/395 [00:11<00:03, 28.65it/s]


Epoch 7:  60%|█████▉    | 1460/2437 [04:10<02:47,  5.84it/s, loss=0.0405, v_num=6, val_loss=0.0794]

In [ ]:
torch.save(net.net.state_dict(), 'dncnn-with-gaussian.pth')

### Salt&Pepper Noise

In [ ]:
net.net.load_state_dict(torch.load('dncnn-noiseless.ckpt'))

In [ ]:
transform = FastMRITransform(
    RandomMaskFunc([0.08], [4]),
    noise_level=5e4,
    noise_type='salt'
)

train_dataset = FastMRIh5Dataset('small_fastmri_pd_3t/train.h5', transform)
val_dataset = FastMRIh5Dataset('small_fastmri_pd_3t/val.h5', transform)
train_generator = torch.utils.data.DataLoader(train_dataset, batch_size=1, num_workers=12)
val_generator = torch.utils.data.DataLoader(val_dataset, batch_size=1, num_workers=12)

In [ ]:
trainer = pl.Trainer(
    gpus=1, max_epochs=20,
    accumulate_grad_batches=32,
    terminate_on_nan=True, 
    default_root_dir='logs/DnCNN_salt',
    callbacks=[
        pl.callbacks.ModelCheckpoint(
            save_last=True,
            save_top_k=4, 
            monitor='val_loss', 
            filename='{epoch}-{ssim:.4f}-{psnr:.4f}-{nmse:.5f}'
        ),
        pl.callbacks.LearningRateMonitor(logging_interval='epoch'),
        pl.callbacks.GPUStatsMonitor(temperature=True)
    ]
);

trainer.fit(net, train_dataloader=train_generator, val_dataloaders=val_generator)

In [ ]:
torch.save(net.net.state_dict(), 'dncnn-with-salt.pth')

### Gaussian + Salt&Pepper Noise

In [ ]:
net.net.load_state_dict(torch.load('dncnn-noiseless.ckpt'))

In [ ]:
transform = FastMRITransform(
    RandomMaskFunc([0.08], [4]),
    noise_level=5e4,
    noise_type='salt'
)

train_dataset = FastMRIh5Dataset('small_fastmri_pd_3t/train.h5', transform)
val_dataset = FastMRIh5Dataset('small_fastmri_pd_3t/val.h5', transform)
train_generator = torch.utils.data.DataLoader(train_dataset, batch_size=1, num_workers=12)
val_generator = torch.utils.data.DataLoader(val_dataset, batch_size=1, num_workers=12)

In [ ]:
trainer = pl.Trainer(
    gpus=1, max_epochs=20,
    accumulate_grad_batches=32,
    terminate_on_nan=True, 
    default_root_dir='logs/DnCNN_salt',
    callbacks=[
        pl.callbacks.ModelCheckpoint(
            save_last=True,
            save_top_k=4, 
            monitor='val_loss', 
            filename='{epoch}-{ssim:.4f}-{psnr:.4f}-{nmse:.5f}'
        ),
        pl.callbacks.LearningRateMonitor(logging_interval='epoch'),
        pl.callbacks.GPUStatsMonitor(temperature=True)
    ]
);

trainer.fit(net, train_dataloader=train_generator, val_dataloaders=val_generator)

In [ ]:
torch.save(net.net.state_dict(), 'dncnn-with-gaussian-salt.pth')